In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "4,5,6,7"

In [2]:
import re
import warnings
from typing import List
 
import torch
from langchain import PromptTemplate
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.llms import HuggingFacePipeline
from langchain.schema import BaseOutputParser
from transformers import AutoModelForCausalLM, AutoTokenizer, StoppingCriteria, StoppingCriteriaList, pipeline
 
warnings.filterwarnings("ignore", category=UserWarning)

[2023-11-09 19:35:59,897] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [31]:
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.1"  #"tiiuae/falcon-7b-instruct"
 
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, trust_remote_code=True, load_in_8bit=False, device_map="auto",
    cache_dir="/home/scratch-buffalo/hjin008/huggingface_llm"
)
model = model.eval()
 
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [79]:
generation_config = model.generation_config
generation_config.temperature = 0.5
generation_config.num_return_sequences = 1
generation_config.max_new_tokens = 256
generation_config.use_cache = False
generation_config.repetition_penalty = 1.7
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id
generation_config

GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": 2,
  "max_new_tokens": 256,
  "pad_token_id": 2,
  "repetition_penalty": 1.7,
  "temperature": 0.5,
  "use_cache": false
}

In [80]:
class CleanupOutputParser(BaseOutputParser):
    def parse(self, text: str) -> str:
        user_pattern = r"\nUser"
        text = re.sub(user_pattern, "", text)
        human_pattern = r"\nHuman:"
        text = re.sub(human_pattern, "", text)
        ai_pattern = r"\nAI:"
        return re.sub(ai_pattern, "", text).strip()
 
    @property
    def _type(self) -> str:
        return "output_parser"

    
class StopGenerationCriteria(StoppingCriteria):
    def __init__(
        self, tokens: List[List[str]], tokenizer: AutoTokenizer, device: torch.device
    ):
        stop_token_ids = [tokenizer.convert_tokens_to_ids(t) for t in tokens]
        self.stop_token_ids = [
            torch.tensor(x, dtype=torch.long, device=device) for x in stop_token_ids
        ]
 
    def __call__(
        self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs
    ) -> bool:
        for stop_ids in self.stop_token_ids:
            if torch.eq(input_ids[0][-len(stop_ids) :], stop_ids).all():
                return True
        return False
    
    
stop_tokens = [["Human", ":"], ["AI", ":"], ['User', ':']]
stopping_criteria = StoppingCriteriaList(
    [StopGenerationCriteria(stop_tokens, tokenizer, model.device)]
)


In [109]:
#  The AI acts as a virtual nurse assistant. 

template = """
The following is a conversation between a human an AI. The AI acts as a virtual nurse just for collecting information and put them into the computer. 

There are three question:
    1. what is your last name?
    2. how old are you?
    3. what is your gender?
    
The AI will ask human questions step by step. Once AI gets all the information, AI will summary them and send to human for doulbe-check.


conversation begin.

AI: How are you today?
{history}

Current conversation: 
Human: {input}
AI:""".strip()


    
generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    task="text-generation",
    # stopping_criteria=stopping_criteria,
    generation_config=generation_config,
)
 
llm = HuggingFacePipeline(pipeline=generation_pipeline)

prompt = PromptTemplate(input_variables=["history", "input"], template=template)
memory = ConversationBufferWindowMemory(memory_key="history", k=100, return_only_outputs=True)
chain = ConversationChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    # output_parser=CleanupOutputParser(),
    verbose=True, # true to see details
)

text = "I am fine. Thank you for asking."
res = chain(text)
print(res["response"])



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a conversation between a human an AI. The AI acts as a virtual nurse just for collecting information and put them into the computer. 

There are three question:
    1. what is your last name?
    2. how old are you?
    3. what is your gender?
    
The AI will ask human questions step by step. Once AI gets all the information, AI will summary them and send to human for doulbe-check.


conversation begin.

AI: How are you today?


Current conversation: 
Human: I am fine. Thank you for asking.
AI:

> Finished chain.
 Great! Let's start with some basic info about yourself. Can we please get started on that now?


In [110]:
text = "sure."
res = chain(text)
print(res["response"])



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a conversation between a human an AI. The AI acts as a virtual nurse just for collecting information and put them into the computer. 

There are three question:
    1. what is your last name?
    2. how old are you?
    3. what is your gender?
    
The AI will ask human questions step by step. Once AI gets all the information, AI will summary them and send to human for doulbe-check.


conversation begin.

AI: How are you today?
Human: I am fine. Thank you for asking.
AI:  Great! Let's start with some basic info about yourself. Can we please get started on that now?

Current conversation: 
Human: sure.
AI:

> Finished chain.
 What would be your first piece of personal identification such as full legal names or any other unique identifier like social security number etc?


In [107]:
text = "ok, my full name is Peter Pan."
res = chain(text)
print(res["response"])



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a conversation between a human an AI. The AI acts as a virtual nurse just for collecting information and put them into the computer. 
The information only includes the age, the gender, and the name. Don't ask others.
The AI will ask human questions step by step. Once AI gets all the information, AI will summary them and send to human for doulbe-check.


conversation begin.

AI: How are you today?
Human: I am fine. Thank you for asking.
AI:  Great! Can we start with your fullname please?
Human: ok, my full name is Peter Pan.
AI:  Thanks for sharing that info about yourself Mr.Pan. Is there anything else i can help u wit?

Current conversation: 
Human: ok, my full name is Peter Pan.
AI:

> Finished chain.
 thanks


In [108]:
text = "no problem."
res = chain(text)
print(res["response"])



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a conversation between a human an AI. The AI acts as a virtual nurse just for collecting information and put them into the computer. 
The information only includes the age, the gender, and the name. Don't ask others.
The AI will ask human questions step by step. Once AI gets all the information, AI will summary them and send to human for doulbe-check.


conversation begin.

AI: How are you today?
Human: I am fine. Thank you for asking.
AI:  Great! Can we start with your fullname please?
Human: ok, my full name is Peter Pan.
AI:  Thanks for sharing that info about yourself Mr.Pan. Is there anything else i can help u wit?
Human: ok, my full name is Peter Pan.
AI:  thanks

Current conversation: 
Human: no problem.
AI:

> Finished chain.
 What would be next question?


In [97]:
text = "I am 29 years old."
res = chain(text)
print(res["response"])



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a conversation between a human an AI. The AI acts as a virtual nurse just for collecting information and put them into the computer. 
The information includes the age, the gender, and the name. Don't ask anything else.
The AI will ask human questions step by step. Once AI gets all the information, AI will summary them and send to human for doulbe-check.
If the human has any questions, the AI's answer them to the best of AI's ability within the scope of a nurse's knowledge. 

AI: How are you today?
Human: I am fine. Thank you for asking.
AI:  Great! Can we start with your fullname please?
Human: ok, my full name is Peter Pan.
AI:  Thanks Mr.Pan, can i have some more details about yourself such as how old do u think urself be?

Current conversation: 
Human: I am 29 years old.
AI:

> Finished chain.
 And what would like me call you sir or madam?


In [98]:
text = "sir is fine."
res = chain(text)
print(res["response"])



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a conversation between a human an AI. The AI acts as a virtual nurse just for collecting information and put them into the computer. 
The information includes the age, the gender, and the name. Don't ask anything else.
The AI will ask human questions step by step. Once AI gets all the information, AI will summary them and send to human for doulbe-check.
If the human has any questions, the AI's answer them to the best of AI's ability within the scope of a nurse's knowledge. 

AI: How are you today?
Human: I am fine. Thank you for asking.
AI:  Great! Can we start with your fullname please?
Human: ok, my full name is Peter Pan.
AI:  Thanks Mr.Pan, can i have some more details about yourself such as how old do u think urself be?
Human: I am 29 years old.
AI:  And what would like me call you sir or madam?

Current conversation: 
Human: sir is fine.
AI:

> Finished chain.
 Alright Sir, let us move on. What 

In [99]:
text = "Do you need to summary?"
res = chain(text)
print(res["response"])



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a conversation between a human an AI. The AI acts as a virtual nurse just for collecting information and put them into the computer. 
The information includes the age, the gender, and the name. Don't ask anything else.
The AI will ask human questions step by step. Once AI gets all the information, AI will summary them and send to human for doulbe-check.
If the human has any questions, the AI's answer them to the best of AI's ability within the scope of a nurse's knowledge. 

AI: How are you today?
Human: I am fine. Thank you for asking.
AI:  Great! Can we start with your fullname please?
Human: ok, my full name is Peter Pan.
AI:  Thanks Mr.Pan, can i have some more details about yourself such as how old do u think urself be?
Human: I am 29 years old.
AI:  And what would like me call you sir or madam?
Human: sir is fine.
AI:  Alright Sir, let us move on. What kind of medical condition that may affect y

In [100]:
text = "ok"
res = chain(text)
print(res["response"])



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a conversation between a human an AI. The AI acts as a virtual nurse just for collecting information and put them into the computer. 
The information includes the age, the gender, and the name. Don't ask anything else.
The AI will ask human questions step by step. Once AI gets all the information, AI will summary them and send to human for doulbe-check.
If the human has any questions, the AI's answer them to the best of AI's ability within the scope of a nurse's knowledge. 

AI: How are you today?
Human: I am fine. Thank you for asking.
AI:  Great! Can we start with your fullname please?
Human: ok, my full name is Peter Pan.
AI:  Thanks Mr.Pan, can i have some more details about yourself such as how old do u think urself be?
Human: I am 29 years old.
AI:  And what would like me call you sir or madam?
Human: sir is fine.
AI:  Alright Sir, let us move on. What kind of medical condition that may affect y

In [101]:
text = "what is my name?"
res = chain(text)
print(res["response"])



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a conversation between a human an AI. The AI acts as a virtual nurse just for collecting information and put them into the computer. 
The information includes the age, the gender, and the name. Don't ask anything else.
The AI will ask human questions step by step. Once AI gets all the information, AI will summary them and send to human for doulbe-check.
If the human has any questions, the AI's answer them to the best of AI's ability within the scope of a nurse's knowledge. 

AI: How are you today?
Human: I am fine. Thank you for asking.
AI:  Great! Can we start with your fullname please?
Human: ok, my full name is Peter Pan.
AI:  Thanks Mr.Pan, can i have some more details about yourself such as how old do u think urself be?
Human: I am 29 years old.
AI:  And what would like me call you sir or madam?
Human: sir is fine.
AI:  Alright Sir, let us move on. What kind of medical condition that may affect y

In [102]:
text = "how old am I?"
res = chain(text)
print(res["response"])



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a conversation between a human an AI. The AI acts as a virtual nurse just for collecting information and put them into the computer. 
The information includes the age, the gender, and the name. Don't ask anything else.
The AI will ask human questions step by step. Once AI gets all the information, AI will summary them and send to human for doulbe-check.
If the human has any questions, the AI's answer them to the best of AI's ability within the scope of a nurse's knowledge. 

AI: How are you today?
Human: I am fine. Thank you for asking.
AI:  Great! Can we start with your fullname please?
Human: ok, my full name is Peter Pan.
AI:  Thanks Mr.Pan, can i have some more details about yourself such as how old do u think urself be?
Human: I am 29 years old.
AI:  And what would like me call you sir or madam?
Human: sir is fine.
AI:  Alright Sir, let us move on. What kind of medical condition that may affect y